In [20]:
import os
import sys
import wandb
sys.path.insert(0,'../')
from libs.ssl_dataloader import *
from libs.ssl_model import *
from libs.ssl_utils import *
from libs import eeg_utils
from sklearn.model_selection import train_test_split
# Use these commands in the same cell to autoreload local library source code
# https://stackoverflow.com/questions/45807760/change-source-code-and-update-jupyter-notebook-but-retain-variables
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
torch.cuda.is_available()

True

In [26]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: dt-young112 (sccn). Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="hbn-ssl", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    #   name=f"experiment_{run}", 
      # Track hyperparameters and run metadata
      config={
        "n_subjects": 500,
        "window": 5,
        'mask_prob':0.3,
        'num_epochs': 10,
        'batch_size': 30,
        'epochs': 10
      })
config = wandb.config

In [16]:
def get_subjects(basedir):
    files = np.array([i for i in os.listdir(basedir) if i.split('.')[-1] == 'set'])
    subjects = np.array([i.split('_')[0] for i in os.listdir(basedir) if i.split('.')[-1] == 'set'])
    return subjects, files

subjects, files = get_subjects("/mnt/nemar/child-mind-rest")
subj_train, subj_test, files_train, files_test = train_test_split(subjects, files, test_size=0.1, random_state=1)
subj_train, subj_val, files_train, files_val = train_test_split(subj_train, files_train, test_size=0.22, random_state=1) # 0.22 x 0.9 = 0.2

In [23]:
SFREQ = 128
dataset = MaskedContrastiveLearningDataset(
        data_dir = "/mnt/nemar/child-mind-rest",
        # subjects = subj_train.tolist(),
        n_subjects=100,
        x_params = {
            'sfreq': SFREQ,
            'window': 2
        },
    )

In [10]:
torch.set_num_threads(1)
print(len(list(torch.utils.data.DataLoader(dataset, num_workers=0))))

8400


In [9]:
torch.set_num_threads(1)
print(len(list(torch.utils.data.DataLoader(dataset, num_workers=24))))

8400


In [11]:
torch.set_num_threads(1)
print(len(list(torch.utils.data.DataLoader(dataset, num_workers=12))))

8400


In [12]:
torch.set_num_threads(1)
print(len(list(torch.utils.data.DataLoader(dataset, num_workers=6))))

8400


In [13]:
torch.set_num_threads(1)
print(len(list(torch.utils.data.DataLoader(dataset, num_workers=3))))

8400


In [14]:
torch.set_num_threads(1)
print(len(list(torch.utils.data.DataLoader(dataset, num_workers=30))))

8400


In [24]:
import time
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, num_workers=0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Wav2VecBrainModel()
model.to(device=device)
model.train()
for e in range(1):
    start_time = time.time()
    for t, (samples, subj) in enumerate(dataloader):
        shuffling_indices = list(range(samples.shape[0]))
        np.random.shuffle(shuffling_indices)
        subj = np.array(subj)[shuffling_indices]
        shuffling_indices = torch.tensor(shuffling_indices)
        samples = torch.index_select(samples, 0, shuffling_indices)

        samples = samples.to(device=device, dtype=torch.float32)
        print(model(samples).shape, subj)
        print('Took %s second(s)' % (time.time() - start_time))
        start_time = time.time()

/home/dung/eeg-ssl/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


torch.Size([128, 768, 52]) ('NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VED', 'NDAREK395BM3', 'NDARFN553LUV', 'NDARYR771VE

KeyboardInterrupt: 

In [7]:
# print('Length of dataset', len(dataset))
print('X dim', dataset[0][0].shape)
print('Y', dataset[0][1])
eeg_utils.plot_raw_eeg(dataset[5][0], 128)

NotImplementedError: Subclasses of Dataset should implement __getitem__.

In [23]:
model = Wav2VecBrainModel()
task = MaskedContrastiveLearningTask(dataset, 
        task_params={
            'mask_prob': config.mask_prob
        },
        train_params={
            'num_epochs': config.epochs,
            'batch_size': config.batch_size,
            'print_every': 100
        }
)
trained_model = task.train(model)

Training on  cuda
Epoch 0, Iteration 0, loss = 3.7844
Epoch 0, Iteration 100, loss = nan
Epoch 0, Iteration 200, loss = nan


KeyboardInterrupt: 